In [1]:
# --RUN--
import numpy as np
import pandas as pd

## Understanding the data

In [2]:
# Understanding the data 
df_1= pd.read_csv('dsc_fc_summed_spectra_2016_v01.csv',
delimiter = ',', parse_dates=[0], 
infer_datetime_format=True, na_values='0',
header = None)

In [3]:
df_1.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,2016-01-01 00:00:00,6.83609,-3.37934,-12.9205,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 00:01:00,6.76732,-3.30194,-12.9967,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01 00:02:00,6.39107,-2.61173,-13.3271,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01 00:03:00,6.44897,-2.61525,-13.3299,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01 00:04:00,6.58758,-2.73082,-13.2361,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Checking for amount of NaN values to select valid processing procedures(such as making decisions about dropping NaN values)
len(df_1.index)-df_1.isnull().sum()

0     283680
1     280417
2     280417
3     280417
4      24467
5      30543
6      45432
7      54624
8      67525
9      79534
10     92055
11    102494
12    115994
13    124868
14    140193
15    148917
16    156121
17    164561
18    172138
19    180264
20    190124
21    195587
22    199324
23    199709
24    202896
25    199364
26    199675
27    195229
28    190569
29    180294
30    173573
31    161682
32    153027
33    137721
34    129946
35    122063
36    113323
37    107108
38     99883
39     89245
40     81933
41     70968
42     65390
43     58596
44     49490
45     39850
46     31554
47     23651
48     20427
49     13621
50     12378
51     10530
52     10338
53      9946
dtype: int64

In [5]:
df_1.columns=["Time","Bx","By","Bz","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50"]

In [6]:
df_1.iloc[0:184][["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50"]]

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
 df_1.iloc[0].isna().any(axis=0)

True

In [8]:
df_1.iloc[0].isna().sum().sum()

50

In [9]:
len(df_1['Time'])

283680

## Filtering The Useful Data

We noticed that the kp values are determined every 3 hours and 3 minutes, which is equivalent to 183 minutes. Our strategy is to use the entire set of 183 data points to predict the kp value for that specific time period. This approach helps mitigate the impact of potential errors in the data due to sensitivity loss. By utilizing the complete set of data points, any errors in individual data points are balanced out by the overall window, allowing the model to better ignore the anomalies and improve its learning process.

In [1]:
# If the count of missing values in that column is greater than a threshold (total rows - 3),
# it implies that a substantial portion of the column is missing
df_final = pd.DataFrame()
df_dup=df_1
a=int(len(df_1['Time'])/183)+1
for i in range(1,a):
    k=183
    c=(k+1)*i
    df=df_1.iloc[c:c+k]
    try:
        g=0
        for j in range(1,50):
            if(df[str(j)].isna().sum()>(len(df["Time"])-3)):
                g=1
                print(c,c+k+1,j)
                break
                #df_dup=df_1.drop(range(c,c+k+1))
                #print(c,c+k+1)
                #break
        if(not g):
            df_final=pd.concat([df_final,df],axis=0)
    except:
        break

NameError: name 'pd' is not defined

In [21]:
len(df_final["Time"])
# Number of remaining rows

9699

In [22]:
len(df_1["Time"])

283680

## Merged Data

In [2]:
# --RUN--
# Loading in the dataset where all the data of different years has been merged and attached with Kp values such that only 
# One in 183 values has an actual value(Here the first row of the window)
full_data=pd.read_csv('merged.csv')

In [24]:
# --RUN--
full_data.iloc[365:368]

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
365,2016-01-01 06:05:00,4.29098,-6.44710,-7.19206,143.834,109.560,125.475,124.1160,127.1630,75.4192,...,149.776,152.936,135.228,161.165,140.635,NaN,NaN,NaN,NaN,NaN
366,2016-01-01 06:06:00,3.27720,-2.49201,-9.90422,148.235,104.279,121.785,90.3689,89.4646,72.6499,...,146.422,158.682,139.984,157.239,141.956,NaN,NaN,NaN,NaN,NaN
367,2016-01-01 06:07:00,2.27488,-2.47778,-10.35640,143.336,103.440,126.416,130.6150,127.0710,72.6778,...,150.588,155.391,132.879,161.134,141.838,2016.0,1.0,1.0,30681.25,5.0


In [4]:
# --RUN--
full_data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [5]:
full_data.head(2)

,date,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,year,month,day,days,Kp
0,2016-01-01 00:00:00,6.83609,-3.37934,-12.9205,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 00:01:00,6.76732,-3.30194,-12.9967,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2016.0,1.0,1.0,30681.0,6.0


In [35]:
full_data.columns

Index(['date', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', 'year', 'month', 'day', 'days', 'Kp'],
      dtype='object')

In [6]:
# --RUN--
full_data.columns=["Time","Bx","By","Bz","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50",'year', 'month', 'day', 'days', 'Kp']

In [7]:
full_data

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
0,2016-01-01 00:00:00,6.83609,-3.37934,-12.920500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 00:01:00,6.76732,-3.30194,-12.996700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2016.0,1.0,1.0,30681.0,6.0
2,2016-01-01 00:02:00,6.39107,-2.61173,-13.327100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01 00:03:00,6.44897,-2.61525,-13.329900,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01 00:04:00,6.58758,-2.73082,-13.236100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3277435,2023-05-02 23:55:00,4.33665,2.80127,0.954633,1.004790,0.231726,4.64757,0.626576,3.54595,0.231726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277436,2023-05-02 23:56:00,4.66778,2.10582,1.260660,0.781621,0.231726,5.97222,0.360648,3.39143,0.231726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277437,2023-05-02 23:57:00,4.28322,3.02154,0.927773,0.854185,0.231726,8.10454,2.025800,4.22000,0.231726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277438,2023-05-02 23:58:00,4.31376,2.67727,1.723270,0.801559,0.231726,12.76210,2.818550,4.82242,0.301864,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Preparing X,Y

In [44]:
# --RUN--
y=full_data["Kp"]

In [45]:
# --RUN--
# Taking only the non NaN values of Kp
lol = y[~(pd.isna(y))]

In [47]:
# --RUN--
len(lol)

18208

In [52]:
lol

1          6.000
184        5.333
367        5.000
550        3.333
733        1.667
           ...  
3276550    2.667
3276733    1.000
3276916    1.000
3277099    1.667
3277282    0.667
Name: Kp, Length: 18208, dtype: float64

We noticed that the kp values are determined every 3 hours and 3 minutes, which is equivalent to 183 minutes. Our strategy is to use the entire set of 183 data points to predict the kp value for that specific time period. This approach helps mitigate the impact of potential errors in the data due to sensitivity loss. By utilizing the complete set of data points, any errors in individual data points are balanced out by the overall window, allowing the model to better ignore the anomalies and improve its learning process.

In [54]:
# --RUN--
# Applying the previously used filter with minor adjustments on the entire dataset
# If the count of missing values in that column is greater than a threshold (total rows - 3),
# it implies that a substantial portion of the column is missing
df_final = pd.DataFrame()
a=len(lol.index)
index= lol.index
for i in range(0,a-1):
    # k=183
    c=index[i]
    l=index[i+1]
    df=full_data.iloc[c:l]
    # print("hi ",c)
    try:
        g=0
        for j in range(1,50):
            if(df[str(j)].isna().sum()>(len(df["Time"])-1)):
                g+=1
                # print(c,c+k+1,j)
                #break
                #df_dup=df_1.drop(range(c,c+k+1))
                #print(c,c+k+1)
                #break
        if(g<8):
            #df.interpolate(method='polynomial',order=2)
            df_final=pd.concat([df_final,df],axis=0)
    except Exception as e:
        print(e)
        break

In [55]:
# --RUN--
# The number of usable rows
len(df_final["Time"])

148329

In [33]:
# --RUN--
df_final[["Bx","By","Bz","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50"]]

,Bx,By,Bz,1,2,3,4,5,6,7,...,41,42,43,44,45,46,47,48,49,50
184,3.354850,0.889208,-12.89340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,3.230850,0.825016,-12.88540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,3.336180,0.858471,-12.85350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,3.288260,0.785345,-12.87140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,3.281750,0.790407,-12.85160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270755,-0.236318,0.074649,3.92354,2.39990,0.231726,6.07773,1.481970,7.53440,0.517735,3.34697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270756,0.015343,0.208545,4.23907,3.56489,0.231726,3.91604,1.013590,4.87756,0.536333,5.20108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270757,-0.277844,-0.796585,4.71982,1.05449,0.231726,5.16990,3.945790,7.74981,0.514977,3.57455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270758,0.229992,-0.274262,5.09344,1.08460,0.231726,4.61619,0.923816,6.17177,0.295443,2.24806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# Trial to interpolate using polynomial method
df_final[["50"]].interpolate(method='polynomial',order=5).isna().count()

50    148596
dtype: int64

In [10]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148596 entries, 184 to 3270782
Data columns (total 59 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    148596 non-null  object 
 1   Bx      146735 non-null  float64
 2   By      146735 non-null  float64
 3   Bz      146735 non-null  float64
 4   1       99701 non-null   float64
 5   2       100764 non-null  float64
 6   3       101923 non-null  float64
 7   4       103458 non-null  float64
 8   5       104583 non-null  float64
 9   6       105464 non-null  float64
 10  7       106152 non-null  float64
 11  8       107051 non-null  float64
 12  9       106470 non-null  float64
 13  10      109016 non-null  float64
 14  11      109924 non-null  float64
 15  12      111516 non-null  float64
 16  13      112109 non-null  float64
 17  14      112373 non-null  float64
 18  15      112810 non-null  float64
 19  16      113308 non-null  float64
 20  17      113536 non-null  float64
 21  18     

In [36]:
df_final.loc[365:370]

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
365,2016-01-01 06:05:00,4.29098,-6.44710,-7.19206,143.834,109.560,125.475,124.1160,127.1630,75.4192,...,149.776,152.936,135.228,161.165,140.635,NaN,NaN,NaN,NaN,NaN
366,2016-01-01 06:06:00,3.27720,-2.49201,-9.90422,148.235,104.279,121.785,90.3689,89.4646,72.6499,...,146.422,158.682,139.984,157.239,141.956,NaN,NaN,NaN,NaN,NaN
367,2016-01-01 06:07:00,2.27488,-2.47778,-10.35640,143.336,103.440,126.416,130.6150,127.0710,72.6778,...,150.588,155.391,132.879,161.134,141.838,2016.0,1.0,1.0,30681.25,5.0
368,2016-01-01 06:08:00,2.24868,-3.21964,-10.07340,149.853,107.864,119.314,84.1787,92.3789,73.7542,...,149.614,152.029,138.220,162.124,138.485,NaN,NaN,NaN,NaN,NaN
369,2016-01-01 06:09:00,2.21823,-3.59131,-10.00470,155.906,100.759,115.752,134.5400,139.4980,104.3230,...,142.868,161.760,145.331,154.375,143.916,NaN,NaN,NaN,NaN,NaN
370,2016-01-01 06:10:00,2.29291,-3.81151,-9.93845,144.020,106.074,124.187,128.5960,130.6200,72.7200,...,145.815,153.230,137.637,159.079,137.421,NaN,NaN,NaN,NaN,NaN


In [56]:
# --RUN--
# Amongst various trials We realized it would be best to fill the NaN values to (mean+ Gaussian Noise)
def fillnanVals(df):
    cols = ["Bx","By","Bz","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50"]
    for i in cols:
        print(i)
        g= df[i]
        mean = g.mean()
        std = np.sqrt(g.var())
        for j in range(len(g)):
            if(np.isnan(g.iloc[j])):
                # g.iloc[j]=np.random.normal(mean,std)
                g.iloc[j]=mean+np.random.normal(0,1,1)
            

In [38]:
full_copy=full_data.copy()

In [39]:
fillnanVals(full_copy)

Bx


/tmp/ipykernel_914102/1722576194.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g.iloc[j]=mean+np.random.normal(0,1,1)


By


KeyboardInterrupt: 

In [14]:
# Saving the csv file
full_copy.to_csv("Interpolated.csv")

In [16]:
full_copy.head(10)

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
0,2016-01-01 00:00:00,6.83609,-3.37934,-12.9205,13.822494,2.242636,213.400058,-6.892206,175.553311,296.638701,...,196.712804,417.837842,339.230905,374.268334,124.576007,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 00:01:00,6.76732,-3.30194,-12.9967,9.772525,27.568609,-11.249177,211.273846,187.817035,213.511386,...,313.299907,208.648229,299.434847,304.549578,403.258915,2016.0,1.0,1.0,30681.0,6.0
2,2016-01-01 00:02:00,6.39107,-2.61173,-13.3271,9.312870,-6.126659,207.555304,-60.654741,151.427363,52.992652,...,286.266360,435.465582,158.407105,444.477576,211.843137,NaN,NaN,NaN,NaN,NaN
3,2016-01-01 00:03:00,6.44897,-2.61525,-13.3299,41.998184,39.920549,193.203909,-65.324284,100.413238,121.455412,...,335.672961,342.285371,228.034150,213.140797,358.551864,NaN,NaN,NaN,NaN,NaN
4,2016-01-01 00:04:00,6.58758,-2.73082,-13.2361,92.080402,22.734833,58.141349,203.945116,92.017638,167.390414,...,306.626214,431.378043,670.606734,324.877475,220.484336,NaN,NaN,NaN,NaN,NaN
5,2016-01-01 00:05:00,6.44875,-2.68868,-13.3076,219.664898,24.315339,209.620023,146.259880,86.910019,277.691854,...,199.426909,333.885421,360.105470,500.308295,244.883219,NaN,NaN,NaN,NaN,NaN
6,2016-01-01 00:06:00,6.38849,-2.61678,-13.3695,114.129126,94.177282,79.589791,-0.310749,170.133065,99.174240,...,371.478220,524.068497,259.455199,371.458058,110.910516,NaN,NaN,NaN,NaN,NaN
7,2016-01-01 00:07:00,6.28842,-2.54100,-13.4081,86.753240,-38.957609,141.381418,11.714234,-94.411133,136.338699,...,410.762896,190.576235,234.720604,374.284907,145.716600,NaN,NaN,NaN,NaN,NaN
8,2016-01-01 00:08:00,6.32911,-2.58715,-13.3603,6.326401,26.182327,78.901281,118.256865,42.960806,181.297205,...,370.814779,378.194594,361.962501,494.079523,373.246491,NaN,NaN,NaN,NaN,NaN
9,2016-01-01 00:09:00,6.33381,-2.67437,-13.3147,9.296030,64.013005,25.883433,-45.290407,151.483111,258.024999,...,116.447743,371.304728,434.615571,126.288004,427.043803,NaN,NaN,NaN,NaN,NaN


In [26]:
df_final.loc[360:370]

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
360,2016-01-01 06:00:00,4.27908,-7.85653,-6.08907,142.838,103.9860,130.149,131.5390,125.0320,104.7840,...,149.299,155.784,134.033,160.265,145.540,NaN,NaN,NaN,NaN,NaN
361,2016-01-01 06:01:00,4.12715,-7.86097,-6.10480,146.516,110.2910,124.393,123.6640,131.0430,112.5760,...,149.229,151.314,137.837,164.427,139.397,NaN,NaN,NaN,NaN,NaN
362,2016-01-01 06:02:00,4.32586,-8.00896,-5.69739,137.917,99.0083,132.370,135.8600,123.7810,101.3080,...,149.125,159.795,135.983,158.750,146.435,NaN,NaN,NaN,NaN,NaN
363,2016-01-01 06:03:00,4.37219,-7.85818,-5.82984,144.509,104.1990,125.531,132.3080,128.6280,104.2120,...,149.844,155.673,136.598,160.285,141.758,NaN,NaN,NaN,NaN,NaN
364,2016-01-01 06:04:00,4.58858,-7.81308,-5.78437,143.500,102.3960,128.116,133.3950,126.4230,102.3280,...,145.818,158.667,136.654,155.294,168.134,NaN,NaN,NaN,NaN,NaN
365,2016-01-01 06:05:00,4.29098,-6.44710,-7.19206,143.834,109.5600,125.475,124.1160,127.1630,75.4192,...,149.776,152.936,135.228,161.165,140.635,NaN,NaN,NaN,NaN,NaN
366,2016-01-01 06:06:00,3.27720,-2.49201,-9.90422,148.235,104.2790,121.785,90.3689,89.4646,72.6499,...,146.422,158.682,139.984,157.239,141.956,NaN,NaN,NaN,NaN,NaN
368,2016-01-01 06:08:00,2.24868,-3.21964,-10.07340,149.853,107.8640,119.314,84.1787,92.3789,73.7542,...,149.614,152.029,138.220,162.124,138.485,NaN,NaN,NaN,NaN,NaN
369,2016-01-01 06:09:00,2.21823,-3.59131,-10.00470,155.906,100.7590,115.752,134.5400,139.4980,104.3230,...,142.868,161.760,145.331,154.375,143.916,NaN,NaN,NaN,NaN,NaN
370,2016-01-01 06:10:00,2.29291,-3.81151,-9.93845,144.020,106.0740,124.187,128.5960,130.6200,72.7200,...,145.815,153.230,137.637,159.079,137.421,NaN,NaN,NaN,NaN,NaN


## Saving the Preproccessed Data

In [57]:
# --RUN--
final_copy=df_final.copy()

In [58]:
# --RUN--
fillnanVals(final_copy)

Bx


/tmp/ipykernel_914102/1722576194.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g.iloc[j]=mean+np.random.normal(0,1,1)


By
Bz
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [60]:
# --RUN--
final_copy.to_csv("Filtered_Interpolation.csv")
# Saving the csv file with Interpolation and Kp

In [25]:
# --RUN--
final_copy=final_copy.reset_index(drop=True)

In [42]:
# --RUN--
final_copy

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
184,2016-01-01 03:04:00,3.354850,0.889208,-12.89340,66.019119,30.376091,81.067780,67.816825,81.634751,69.161085,...,318.592158,311.481679,288.522293,327.790527,286.270825,2016.0,1.0,1.0,30681.125,5.333
185,2016-01-01 03:05:00,3.230850,0.825016,-12.88540,69.161614,25.619158,80.737362,67.101316,80.879001,68.554295,...,317.847774,313.183389,286.668015,329.756261,287.303915,NaN,NaN,NaN,NaN,NaN
186,2016-01-01 03:06:00,3.336180,0.858471,-12.85350,68.255955,27.445076,79.321671,69.063314,81.211808,67.830480,...,318.963260,313.541573,286.072561,327.885022,288.724639,NaN,NaN,NaN,NaN,NaN
187,2016-01-01 03:07:00,3.288260,0.785345,-12.87140,67.362437,26.570600,79.445221,67.529253,82.390888,68.326827,...,319.279778,311.099937,287.074361,328.699679,286.769187,NaN,NaN,NaN,NaN,NaN
188,2016-01-01 03:08:00,3.281750,0.790407,-12.85160,66.345401,27.383841,78.718984,68.274730,82.063167,66.876713,...,316.638880,313.122990,287.544693,329.899069,285.561021,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270755,2023-04-28 08:35:00,-0.236318,0.074649,3.92354,2.399900,0.231726,6.077730,1.481970,7.534400,0.517735,...,318.143275,313.344562,286.036226,329.821362,288.541344,NaN,NaN,NaN,NaN,NaN
3270756,2023-04-28 08:36:00,0.015343,0.208545,4.23907,3.564890,0.231726,3.916040,1.013590,4.877560,0.536333,...,316.416434,313.350570,288.209974,329.314539,287.998174,NaN,NaN,NaN,NaN,NaN
3270757,2023-04-28 08:37:00,-0.277844,-0.796585,4.71982,1.054490,0.231726,5.169900,3.945790,7.749810,0.514977,...,318.738726,311.986214,287.069977,328.293408,287.828746,NaN,NaN,NaN,NaN,NaN
3270758,2023-04-28 08:38:00,0.229992,-0.274262,5.09344,1.084600,0.231726,4.616190,0.923816,6.171770,0.295443,...,318.864576,311.682694,288.676713,329.198478,286.331623,NaN,NaN,NaN,NaN,NaN


In [11]:
final_copy1=df_final.copy()
final_copy1

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
184,2016-01-01 03:04:00,3.354850,0.889208,-12.89340,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2016.0,1.0,1.0,30681.125,5.333
185,2016-01-01 03:05:00,3.230850,0.825016,-12.88540,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,2016-01-01 03:06:00,3.336180,0.858471,-12.85350,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,2016-01-01 03:07:00,3.288260,0.785345,-12.87140,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,2016-01-01 03:08:00,3.281750,0.790407,-12.85160,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270778,2023-04-28 08:58:00,-2.153650,-2.794570,2.74216,2.10516,0.231726,8.66518,1.66949,9.60619,0.674678,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270779,2023-04-28 08:59:00,-0.530506,-1.221680,1.90248,3.53354,0.231726,7.94703,2.72486,6.44502,0.625266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270780,2023-04-28 09:00:00,2.123870,2.115340,2.90622,2.03191,0.231726,8.19133,4.13665,9.29280,0.536912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270781,2023-04-28 09:01:00,1.658880,1.733010,3.50017,2.67550,0.231726,5.25048,2.57350,7.95142,0.478097,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
fillnanVals(final_copy1)

Bx


/tmp/ipykernel_914102/1722576194.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g.iloc[j]=mean+np.random.normal(0,1,1)


By
Bz
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [13]:
final_copy1

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
184,2016-01-01 03:04:00,3.354850,0.889208,-12.89340,67.661129,26.298362,80.341246,67.560987,81.205316,69.188238,...,318.558462,312.474566,287.166337,326.787492,288.300935,2016.0,1.0,1.0,30681.125,5.333
185,2016-01-01 03:05:00,3.230850,0.825016,-12.88540,68.222121,26.569572,81.661173,67.687720,82.946629,68.329891,...,318.539435,312.626192,286.138829,330.584637,285.012897,NaN,NaN,NaN,NaN,NaN
186,2016-01-01 03:06:00,3.336180,0.858471,-12.85350,68.116187,28.238115,79.188128,67.636326,81.433154,69.405840,...,319.770807,311.872892,286.909250,328.297530,286.461641,NaN,NaN,NaN,NaN,NaN
187,2016-01-01 03:07:00,3.288260,0.785345,-12.87140,68.169848,27.588688,79.770938,68.573323,79.634808,67.530203,...,318.322704,311.853706,288.076535,327.849997,287.781781,NaN,NaN,NaN,NaN,NaN
188,2016-01-01 03:08:00,3.281750,0.790407,-12.85160,67.638814,26.372656,81.178891,68.502857,80.091303,68.529303,...,318.339353,312.271543,286.239488,326.202074,284.991512,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270778,2023-04-28 08:58:00,-2.153650,-2.794570,2.74216,2.105160,0.231726,8.665180,1.669490,9.606190,0.674678,...,317.628831,310.507287,286.681660,328.060912,286.769763,NaN,NaN,NaN,NaN,NaN
3270779,2023-04-28 08:59:00,-0.530506,-1.221680,1.90248,3.533540,0.231726,7.947030,2.724860,6.445020,0.625266,...,319.478784,312.476512,285.524121,327.828499,286.116333,NaN,NaN,NaN,NaN,NaN
3270780,2023-04-28 09:00:00,2.123870,2.115340,2.90622,2.031910,0.231726,8.191330,4.136650,9.292800,0.536912,...,317.416048,311.160027,287.001423,327.457420,286.727591,NaN,NaN,NaN,NaN,NaN
3270781,2023-04-28 09:01:00,1.658880,1.733010,3.50017,2.675500,0.231726,5.250480,2.573500,7.951420,0.478097,...,319.263281,311.440987,287.279124,327.830426,287.227242,NaN,NaN,NaN,NaN,NaN


In [14]:
final_copy1.to_csv("Filtered_Interpolation1.csv")

In [16]:
final_copy1['Kp'].isna().sum()

147773

In [17]:
final_copy1

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
184,2016-01-01 03:04:00,3.354850,0.889208,-12.89340,67.661129,26.298362,80.341246,67.560987,81.205316,69.188238,...,318.558462,312.474566,287.166337,326.787492,288.300935,2016.0,1.0,1.0,30681.125,5.333
185,2016-01-01 03:05:00,3.230850,0.825016,-12.88540,68.222121,26.569572,81.661173,67.687720,82.946629,68.329891,...,318.539435,312.626192,286.138829,330.584637,285.012897,NaN,NaN,NaN,NaN,NaN
186,2016-01-01 03:06:00,3.336180,0.858471,-12.85350,68.116187,28.238115,79.188128,67.636326,81.433154,69.405840,...,319.770807,311.872892,286.909250,328.297530,286.461641,NaN,NaN,NaN,NaN,NaN
187,2016-01-01 03:07:00,3.288260,0.785345,-12.87140,68.169848,27.588688,79.770938,68.573323,79.634808,67.530203,...,318.322704,311.853706,288.076535,327.849997,287.781781,NaN,NaN,NaN,NaN,NaN
188,2016-01-01 03:08:00,3.281750,0.790407,-12.85160,67.638814,26.372656,81.178891,68.502857,80.091303,68.529303,...,318.339353,312.271543,286.239488,326.202074,284.991512,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270778,2023-04-28 08:58:00,-2.153650,-2.794570,2.74216,2.105160,0.231726,8.665180,1.669490,9.606190,0.674678,...,317.628831,310.507287,286.681660,328.060912,286.769763,NaN,NaN,NaN,NaN,NaN
3270779,2023-04-28 08:59:00,-0.530506,-1.221680,1.90248,3.533540,0.231726,7.947030,2.724860,6.445020,0.625266,...,319.478784,312.476512,285.524121,327.828499,286.116333,NaN,NaN,NaN,NaN,NaN
3270780,2023-04-28 09:00:00,2.123870,2.115340,2.90622,2.031910,0.231726,8.191330,4.136650,9.292800,0.536912,...,317.416048,311.160027,287.001423,327.457420,286.727591,NaN,NaN,NaN,NaN,NaN
3270781,2023-04-28 09:01:00,1.658880,1.733010,3.50017,2.675500,0.231726,5.250480,2.573500,7.951420,0.478097,...,319.263281,311.440987,287.279124,327.830426,287.227242,NaN,NaN,NaN,NaN,NaN


In [18]:
final_copy1.to_csv("Filtered_Interpolation1.csv")